In [1]:
%load_ext autoreload
%autoreload 2

from text3d2video.clip_metrics import CLIPMetrics
import wandb
from omegaconf import OmegaConf
from text3d2video.evaluation.video_comparison import (
    VideoLabel,
    make_comparison_vid,
    scene_key_fun,
)

In [2]:
metrics = CLIPMetrics()

In [3]:
from text3d2video.generative_rendering.configs import (
    NoiseInitializationMethod,
    RunGenerativeRenderingConfig,
)
from text3d2video.util import group_list_by

api = wandb.Api()

filters = {"tags": "gr_ablation_2"}
runs = list(api.runs("romeu/diffusion-3D-features", filters=filters))


def sort_fun(run):
    cfg: RunGenerativeRenderingConfig = OmegaConf.create(run.config)
    gr_config = cfg.generative_rendering
    noise_method = gr_config.noise_initialization.method

    random_noise = noise_method == NoiseInitializationMethod.RANDOM.value
    fixed_noise = noise_method == NoiseInitializationMethod.FIXED.value
    uv_noise = noise_method == NoiseInitializationMethod.UV.value
    uv_noise_pre_attn = uv_noise and gr_config.do_pre_attn_injection
    uv_noise_full_attn = uv_noise_pre_attn and gr_config.do_post_attn_injection

    if uv_noise_full_attn:
        return 0
    if uv_noise_pre_attn:
        return 1
    if uv_noise:
        return 2
    if fixed_noise:
        return 3
    if random_noise:
        return 4

    else:
        raise ValueError(
            f"Unknown config: noise={gr_config.noise_initialization.method}, pre_attn={gr_config.do_pre_attn_injection}, post_attn={gr_config.do_post_attn_injection}"
        )


# group runs by scene
runs_grouped = group_list_by(runs, scene_key_fun)

# sort runs in each scene consistently
for runs in runs_grouped:
    runs.sort(key=sort_fun)

In [4]:
def info_fun(run, _):
    label = VideoLabel()
    cfg = OmegaConf.create(run.config)
    gr_config = cfg.generative_rendering
    noise_method = gr_config.noise_initialization.method

    random_noise = noise_method == NoiseInitializationMethod.RANDOM.value
    fixed_noise = noise_method == NoiseInitializationMethod.FIXED.value
    uv_noise = noise_method == NoiseInitializationMethod.UV.value
    uv_noise_pre_attn = uv_noise and gr_config.do_pre_attn_injection
    uv_noise_full_attn = uv_noise_pre_attn and gr_config.do_post_attn_injection

    if uv_noise_full_attn:
        label.content = "UV noise\nCross-Attn\nFeature Injection"
        return label
    if uv_noise_pre_attn:
        label.content = "UV noise\nCross-Attn"
        return label
    if uv_noise:
        label.content = "UV noise"
        return label
    if fixed_noise:
        label.content = "Fixed noise"
        return label
    if random_noise:
        label.content = "Random noise"
        return label

    return label


def metrics_fun(_, video):
    label = VideoLabel()

    frame_consistency = metrics.frame_consistency(video)
    prompt_fidelity = metrics.prompt_fidelity(video)

    label.content += f"Frame consistency: {frame_consistency:.2f}"
    label.content += f"\nPrompt fidelity: {prompt_fidelity:.2f}"

    return label


def comparison_vid(runs, download=False):
    r0_conf = OmegaConf.create(runs[0].config)
    prompt = r0_conf.prompt
    title = f'Prompt: "{prompt}"'

    vid = make_comparison_vid(
        [runs],
        info_fun_bottom=info_fun,
        download=download,
        title=title,
        show_guidance_video=False,
    )

    return vid

In [5]:
vid = comparison_vid(runs_grouped[0], download=True)
vid.ipython_display()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb:   1 of 1 files downloaded.  


OSError: MoviePy Error: creation of None failed because of the following error:

convert-im6.q16: attempt to perform an operation not allowed by the security policy `@/tmp/tmpx9f2t5nr.txt' @ error/property.c/InterpretImageProperties/3706.
convert-im6.q16: no images defined `PNG32:/tmp/tmp91ew5xif.png' @ error/convert.c/ConvertImageCommand/3229.
.

.This error can be due to the fact that ImageMagick is not installed on your computer, or (for Windows users) that you didn't specify the path to the ImageMagick binary in file conf.py, or that the path you specified is incorrect

In [42]:
from text3d2video.artifacts.animation_artifact import AnimationArtifact
import text3d2video.wandb_util as wbu

r = runs_grouped[0][0]

animation = wbu.first_used_artifact_of_type(r, "animation")
animation = AnimationArtifact.from_wandb_artifact(animation, download=True)

wandb:   2 of 2 files downloaded.  


In [44]:
n_frames = OmegaConf.create(r.config).animation.n_frames
frame_nums = animation.frame_nums(n_frames)

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

: 

In [22]:
from pathlib import Path
import shutil

results_dir = Path("results/gr_ablation/")

if results_dir.exists():
    shutil.rmtree(results_dir)
results_dir.mkdir(parents=True, exist_ok=True)

vid.write_videofile(str(results_dir / "gr_ablation.mp4"))

Moviepy - Building video results/gr_ablation/gr_ablation.mp4.
Moviepy - Writing video results/gr_ablation/gr_ablation.mp4



Moviepy - Done !
Moviepy - video ready results/gr_ablation/gr_ablation.mp4
